In [68]:
import jieba
from collections import Counter
import math
#from tabulate import tabulate

In [7]:
train_pos= '/tensorflow/pos_train.txt'
train_neg= '/tensorflow/neg_train.txt'
test_pos= '/tensorflow/pos_test.txt'
test_neg= '/tensorflow/neg_test.txt'

In [14]:
#返回一个Counter，里面装有filepath里所有word的词频
def initLM(filepath):
    lm=Counter()
    with open(filepath) as f:
        for line in f:
            words= jieba.cut(line.strip())
            for word in words:
                if word >=u'\u4e00' and word<=u'\u9fff':
                    lm[word]+=1
    return lm

In [61]:
lm_pos=initLM(train_pos)
lm_neg= initLM(train_neg)

In [19]:
#根据Counter里已有的pos和neg中出现每个词的概率来判断整个句子是pos还是neg
def sentenceEmotion(sentence, lm_pos, lm_neg):
    pos_freq= float(0)
    neg_freq= float(0)
    for word in jieba.cut(sentence.strip()):
        if word >=u'\u4e00' and word<=u'\u9fff':
            if word in lm_pos:
                pos_freq += math.log(lm_pos[word])
            elif word in lm_neg:
                neg_freq += math.log(lm_neg[word])
    return pos_freq, neg_freq

In [62]:
#给一个全是（pos||neg）句子的文档，看（有多少能句子被判断为pos，多少被误判为neg||有多少能句子被判断为neg，多少被误判为pos）
def judgeEmotion(testFile):
    line_no=0
    posFromTest=0
    negFromTest=0
    with open(testFile) as f:
            for line in f:
                line_no +=1
                pos, neg= sentenceEmotion(line, lm_pos, lm_neg)
                if pos>neg:
                    posFromTest +=1
                   # print line
                    # print "positive emotion"
                else:
                    negFromTest +=1
                   # print line
                   # print "negative"
    return posFromTest, negFromTest,line_no

In [51]:
TP, FN, TrueTotal = judgeEmotion(test_pos)
print (judgeEmotion(test_pos))

(4947, 18, 4965)


In [52]:
TN, FP, FalseTotal = judgeEmotion(test_neg)
print (judgeEmotion(test_neg))

(5561, 12, 5573)


In [60]:
Accuracy = float((TP+TN))/(TrueTotal+FalseTotal)
ErrorRate = float((FP+FN))/(TrueTotal+FalseTotal)
TruePositiveRate =float(TP)/(TrueTotal)
TrueNegativeRate =float(TN)/(FalseTotal)
Specificity = float(TN)/FalseTotal
Precision = float(TP)/TrueTotal
Prevelance= float(TrueTotal)/(TrueTotal+FalseTotal)

print "Accuracy: %f" % Accuracy
print "ErrorRate: %f" % ErrorRate
print "TruePositiveRate: %f"% TruePositiveRate
print "TrueNegativeRate: %f"% TrueNegativeRate
print "Specificity: %f"% Specificity
print "Precision: %f" %Precision
print "Prevelance: %f" % Prevelance

Accuracy: 0.997153
ErrorRate: 0.002847
TruePositiveRate: 0.996375
TrueNegativeRate: 0.997847
Specificity: 0.997847
Precision: 0.996375
Prevelance: 0.471152
